In [0]:
import math
from pyspark.sql.functions import *
import pyspark.sql.functions as f

In [0]:
#read all the files from filestore
moviesrdd = sc.textFile('/FileStore/tables/plot_summaries.txt')
movies = sc.textFile('/FileStore/tables/movie_metadata.tsv')
#user query file containing search terms
queries = sc.textFile('/FileStore/tables/query.txt').collect()
stopwords = sc.textFile('/FileStore/tables/stopwords.txt')

In [0]:
#document count 
number_of_docs = moviesrdd.count()
number_of_docs

42306

In [0]:
#remove spaces from stopwords file and create a list of stopwords
stopwordsrdd = stopwords.flatMap(lambda x: x.split('\n'))
stopwords_list = stopwordsrdd.collect()

In [0]:
#split plot summary by \t
moviesrdd_sep = moviesrdd.map( lambda x : x.split('\t'))
#map the word 
doc_words = moviesrdd_sep.flatMap(lambda x : [((x[0], i.lower()), 1) for i in x[1].split()])
#remove stopwords
doc_words_sw = doc_words.filter( lambda x : x[0][1] not in stopwords_list)
#summing up tf values for each word
doc_words_reduce = doc_words_sw.reduceByKey(lambda x,y:x+y)
tf_doc = doc_words_reduce.map(lambda x : (x[0][1], (x[0][0], x[1])))
#calculating idf values
idf = doc_words_reduce.map(lambda x: (x[0][1], (x[0][0], x[1], 1)))
idf_map = idf.map(lambda x : (x[0], x[1][2]))
#summing up the number of documents for each word
idf_reduce = idf_map.reduceByKey(lambda x,y : x+y)
idf_doc = idf_reduce.map(lambda x: (x[0], math.log10(number_of_docs/x[1])))
#joining tf and idf values
tf_idf = tf_doc.join(idf_doc)
tf_idf_doc = tf_idf.map(lambda x: (x[1][0][0], (x[0], x[1][0][1], x[1][1], x[1][0][1]*x[1][1] )))
movies_sep = movies.map(lambda x : x.split('\t'))
movies = movies_sep.map(lambda x : (x[0], x[1], x[2])).map(lambda x : (x[0], x[2]))
#mapping with movie names
tf_idf_movies = tf_idf_doc.join(movies)

In [0]:
#print top 10 filtered movie titles based on search given by user
for query in queries:
  print("Searching for  ", query)
  query = query.lower()
  query_terms = query.split(" ")
  if len(query_terms) > 1:
    #calculating tf for query
    query_tf= sc.parallelize(query_terms).map(lambda x : (x,1)).reduceByKey(lambda x,y : x+y)
    idf_words= tf_idf_movies.map(lambda x : (x[1][0][0], (x[1][1], x[1][0][2])))
    #joining tf-idf of movies and query
    query_tf_idf = query_tf.leftOuterJoin(idf_words)
    query_tf_idf = query_tf_idf.map(lambda x : (x[0], 0 if x[1][1] is None  else  x[1][0]*x[1][1][1])).map(lambda x: ( (x[0], x[1])))
    query_doc_tf_idf = tf_idf_movies.map(lambda x: (x[1][0][0], (x[1][1], x[1][0][3]))).join(query_tf_idf).map(lambda x : (x[1][0][0], x[1][0][1] , x[1][1]) )
    query_doc_tf_idf = query_doc_tf_idf.map(lambda x: (x[0], (x[1]*x[2], x[2]*x[2], x[1]*x[1]))).reduceByKey(lambda x,y : ((x[0] + y[0], x[1] + y[1], x[2] + y[2])))
    #Cosine Similarity
    cos_sim = query_doc_tf_idf.map(lambda x: (x[0], x[1][0]/(math.sqrt(x[1][1]) * math.sqrt(x[1][2]) ))).sortBy(keyfunc = lambda x: x[1] ,ascending=False).take(10)
    
  else:
    cos_sim = tf_idf_movies.filter(lambda x : x[1][0][0]== query_terms[0] ).sortBy(keyfunc = lambda x: x[1][0][3] ,ascending=False).map(lambda x : (x[1][1],x[1][0][3] )).take(10)
  for result in cos_sim:
    print(f"Movie Name => {result[0]} ")

Searching for Sleeping

Movie Name => Tops with Pops 
Movie Name => Love That Pup 
Movie Name => Tick Tock Tuckered 
Movie Name => Bushwhacked 
Movie Name => Women on the Edge of a Nervous Breakdown 
Movie Name => La Moustache 
Movie Name => The Fabulous Baker Boys 
Movie Name => Danny, the Champion of the World 
Movie Name => Main Aur Mrs Khanna 
Movie Name => Much Ado About Mousing 
Searching for micheal movie

Movie Name => Ghulami 
Movie Name => Nine 
Movie Name => America's Sweethearts 
Movie Name => Sidehackers 
Movie Name => Teri Meherbaniyan 
Movie Name => The Curse 
Movie Name => The Majestic 
Movie Name => Spiderhole 
Movie Name => The Last Movie 
Movie Name => Hollywood Ending 
Searching for romance

Movie Name => 4 Romance 
Movie Name => Future Cops 
Movie Name => The Great Outdoors 
Movie Name => The English Patient 
Movie Name => Second Fiddle 
Movie Name => The manor of Araucaima 
Movie Name => Hannah and Her Sisters 
Movie Name => The Thief and the Cobbler 
Movie Name => In Bruges 
Movie Name => Nightmare Alley